<a href="https://colab.research.google.com/github/AdamQinwt/Colab_Code/blob/master/xor_cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimator

In [0]:
#!git clone https://github.com/tensorflow/models
#!python models/samples/core/get_started/premade_estimator.py
import tensorflow as tf
import numpy as np
problem='xor'
BATCH_SIZE=4
LEARNING_RATE=0.1
IMG_SIZE=2
DATASET_NAME='/content/drive/My Drive/datasets/'+problem+'.tfrecords'
ROOT='//content/drive/My Drive/'+problem+'/'
CLASS_NUM=2

## input_fn

### train input

In [0]:
def parser_data(serialized_example):
    context, sequence = tf.parse_single_sequence_example(serialized_example,
                      context_features={
                          'features': tf.FixedLenFeature([IMG_SIZE],
                          tf.float32),
                          'label': tf.FixedLenFeature(
                              [1], tf.float32)
                      })
    feat = context['features']
#    feat=tf.reshape(feat,IMG_SHAPE)
    label = context['label']
    label = tf.cast(tf.reshape(label, [1]),tf.int32)
    return feat, label

def train_input_fn(file_name, batch_size):
  """An input function for training"""
  # Convert the inputs to a Dataset.
  dataset = tf.data.TFRecordDataset([file_name])
  # dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  # Shuffle, repeat, and batch the examples.
  dataset = dataset.map(parser_data).shuffle(2).repeat().batch(batch_size)

  # Return the read end of the pipeline.
  return dataset.make_one_shot_iterator().get_next()

### test(eval) input

In [0]:
def test_input_fn(file_name, batch_size):
  """An input function for training"""
  # Convert the inputs to a Dataset.
  dataset = tf.data.TFRecordDataset([file_name])
  # dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  # Shuffle, repeat, and batch the examples.
  dataset = dataset.map(parser_data).shuffle(2).batch(batch_size)

  # Return the read end of the pipeline.
  return dataset.make_one_shot_iterator().get_next()

## model_fn

### create_model

In [0]:
def create_model():
  l = tf.keras.layers
  return tf.keras.Sequential(
      [
          l.Dense(3, activation=tf.nn.sigmoid),
          l.Dense(CLASS_NUM)
      ])

### define the estimator functions

In [0]:
def model_fn(features, labels, mode, params):
  """The model_fn argument for creating an Estimator."""
  model = create_model()
  image = features
  if isinstance(image, dict):
    image = features['image']

  if mode == tf.estimator.ModeKeys.PREDICT:
    logits = model(image, training=False)
    predictions = {
        'classes': tf.argmax(logits, axis=1),
        'probabilities': tf.nn.softmax(logits),
    }
    return tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.PREDICT,
        predictions=predictions,
        export_outputs={
            'classify': tf.estimator.export.PredictOutput(predictions)
        })
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)

    logits = model(image, training=True)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    accuracy = tf.metrics.accuracy(
        labels=labels, predictions=tf.argmax(logits, axis=1))
    # Name tensors to be logged with LoggingTensorHook.
    tf.identity(LEARNING_RATE, 'learning_rate')
    tf.identity(loss, 'cross_entropy')
    tf.identity(accuracy[1], name='train_accuracy')
    # Save accuracy scalar to Tensorboard output.
    tf.summary.scalar('train_accuracy', accuracy[1])

    return tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.TRAIN,
        loss=loss,
        train_op=optimizer.minimize(loss, tf.train.get_or_create_global_step()))
  if mode == tf.estimator.ModeKeys.EVAL:
    logits = model(image, training=False)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    return tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.EVAL,
        loss=loss,
        eval_metric_ops={
            'accuracy':
                tf.metrics.accuracy(
                    labels=labels, predictions=tf.argmax(logits, axis=1)),
        })

# train

In [15]:
classifier = tf.estimator.Estimator(
        model_fn=model_fn,
        params={
            'data_format':'channels_last'
        },
        model_dir=ROOT)
# for i in range(10):
t=classifier.train(
    input_fn=lambda:train_input_fn(DATASET_NAME,BATCH_SIZE),
    steps=2000)
print('train')


TypeError: ignored

In [8]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:test_input_fn(DATASET_NAME,BATCH_SIZE)
    )
print(eval_result)

{'accuracy': 1.0, 'loss': 1.9430798e-05, 'global_step': 4000}


In [14]:
predictions = classifier.predict(
    input_fn=lambda:test_input_fn(DATASET_NAME,
    batch_size=1))
for p in predictions:
  print(p)
#print(predictions)

{'classes': 1, 'probabilities': array([1.3469916e-05, 9.9998653e-01], dtype=float32)}
{'classes': 0, 'probabilities': array([9.9995530e-01, 4.4651082e-05], dtype=float32)}
{'classes': 1, 'probabilities': array([1.8630824e-05, 9.9998140e-01], dtype=float32)}
{'classes': 0, 'probabilities': array([9.9999905e-01, 9.9239662e-07], dtype=float32)}


In [10]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [0]:
# clear
# !rm -r /tmp/tmp*